# BERT + Tree Model
- **BERT (Sentence-Transformers)** → Title + Tags + Description embedding
- **Tree-based Gradient Boosting** → Numeric + BERT features
- **Non-linear + semantic aware**

Bu yaklaşım Kaggle Grandmaster ve production sistemlerde kullanılır.
&nbsp;
**NOT**: Bu notebook finale kadar bir çok değişikliğe uğrayacak ve daha kaliteli ve geliştirilmiş modeller hakkında çalışmalar devam edecek.

In [1]:
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
from lightgbm import LGBMClassifier


C:\Users\mahmi\Desktop\VMP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("../Dataset/Final_Dataset.csv")

df = df[[
    "categoryId",
    "title",
    "tags",
    "description",
    "view_count",
    "likes",
    "comment_count",
    "is_trending"
]]

df[["title", "tags", "description"]] = df[["title", "tags", "description"]].fillna("")
for c in ["view_count", "likes", "comment_count"]:
    df[c] = np.log1p(df[c])

df.head()


C:\Users\mahmi\AppData\Local\Temp\ipykernel_3724\3321156476.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Dataset/Final_Dataset.csv")


,categoryId,title,tags,description,view_count,likes,comment_count,is_trending
0,22.0,college farewell video#trending #viralshorts,"college farewell video, DDU farewell video, fa...",DDU farewell party video \n college farewell v...,18.650077,14.258386,7.367709,1
1,10.0,Busking in Manchester 🇬🇧 #blindfaith #guitar #...,,,18.366689,13.738122,8.674197,1
2,22.0,This is what happens when you play Star Wars i...,,Full video : https://www.youtube.com/watch?v=Q...,17.941771,13.880749,8.324821,1
3,10.0,LISA - FUTW (YouTube Music Nights Special Stag...,"Blackpink, Lisa, Music, Fashion, K-Pop, kpop, ...","LISA’s debut album, ‘Alter Ego’ is out now. Li...",16.493511,13.029847,9.923143,1
4,10.0,Uljhi hai yeh kis jaal me tu…. Bengaluru ❤️,,,17.081459,12.725664,7.755339,1


In [3]:
bert = SentenceTransformer("all-MiniLM-L6-v2")

def build_text(row):
    return row["title"] + " [SEP] " + row["tags"] + " [SEP] " + row["description"]

texts = df.apply(build_text, axis=1).tolist()

embeddings = bert.encode(
    texts,
    batch_size=32,
    show_progress_bar=True
)

embeddings.shape


Batches: 100%|██████████| 12189/12189 [15:06<00:00, 13.45it/s] 


(390043, 384)

In [4]:
num_features = df[["categoryId", "view_count", "likes", "comment_count"]].values

X = np.hstack([embeddings, num_features])
y = df["is_trending"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [5]:
model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=128,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight="balanced",
    random_state=42
)

model.fit(X_train, y_train)
print("Model eğitildi")


[LightGBM] [Info] Number of positive: 137345, number of negative: 174689
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.489424 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 98702
[LightGBM] [Info] Number of data points in the train set: 312034, number of used features: 388
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Model eğitildi 🚀


In [6]:
probs = model.predict_proba(X_test)[:,1]
preds = (probs > 0.5).astype(int)

print("ROC-AUC:", roc_auc_score(y_test, probs))
print(classification_report(y_test, preds))


C:\Users\mahmi\Desktop\VMP\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


ROC-AUC: 0.9971911691590913
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     43673
           1       0.99      0.97      0.98     34336

    accuracy                           0.98     78009
   macro avg       0.98      0.98      0.98     78009
weighted avg       0.98      0.98      0.98     78009



In [57]:
def predict_trending_probability(
    categoryId, title, tags, description, views, likes, comments
):
    text = title + " [SEP] " + tags + " [SEP] " + description
    emb = bert.encode([text])

    num = np.array([[
        categoryId,
        np.log1p(views),
        np.log1p(likes),
        np.log1p(comments)
    ]])

    X = np.hstack([emb, num])
    return model.predict_proba(X)[0][1]


# TEST
p = predict_trending_probability(
    10,
    "INSANE Live Vocal Performance",
    "live singing viral performance",
    "Crowd shocked by insane vocals",
    1500,
    700,
    100
)

p1 = predict_trending_probability(
    20,
    "Fortnite new mode gameplay",
    "Amazing gameplay",
    "Fortnite gaming new_mode",
    150000,
    15000,
    5000
)

print(f"Trend olma olasılığı: %{p*100:.2f}")
print(f"Trend olma olasılığı: %{p1*100:.2f}")


Trend olma olasılığı: %0.28
Trend olma olasılığı: %92.16


C:\Users\mahmi\Desktop\VMP\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\mahmi\Desktop\VMP\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


## Model Çıktılarının Yorumlanması

### Music Videosu – Neden Düşük?

- Views: **1,500**
- Likes: **700**
- Comments: **100**
- Kategori: **Music**

Music kategorisi çok rekabetçi.
Etkileşim sayıları trend eşiğinin çok altında kaldığı için,
BERT içeriği anlamasına rağmen model düşük olasılık vermiştir.

Dil iyi ama etkileşim trend için erken


### Gaming Videosu – Neden Yüksek?

- Views: **150,000**
- Likes: **15,000**
- Comments: **5,000**
- Kategori: **Gaming**

- Like/View oranı yüksek
- Comment sayısı güçlü
- Gaming kategorisinde trend eşiği daha düşük
- BERT, “gameplay”, “new mode” gibi trend tetikleyicileri algılamıştır

Hem içerik hem etkileşim trende çok yakın

### Önemli Not

Bu model şu soruya cevap verir:

Bu video, geçmişte trend olmuş videolara ne kadar benziyor?

Erken aşamadaki videoların düşük skor alması normaldir.

### Modelin Kullandığı Ana Sinyaller

- Log(Views, Likes, Comments)
- BERT semantik anlam
- Kategori rekabet seviyesi
- Viral dil + engagement eşikleri
